In [92]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [93]:
# Read sheets from excel file

COL_DATE = "Date"
COL_HOURS = [f"Values_Hour{i:02}" for i in range(1, 25)]

def read_each_sheet(sheet_name, excel_data, plant_name: str=None, index_col=0):

    df_data = excel_data.parse(sheet_name, index_col=index_col)
    
    if plant_name:
        df_data = df_data[df_data["Values_code"] == plant_name]
        df_data.drop(columns=["Values_code"], inplace=True)

    if "Id" in df_data.columns:
        df_data.drop(columns=["Id"], inplace=True)
        
    df_data[COL_DATE] = pd.to_datetime(df_data[COL_DATE])
    # remove days lower than 2024-03-08
    df_data = df_data[df_data[COL_DATE] >= pd.Timestamp("2024-03-08")]
    # df_data.set_index(COL_DATE, inplace=True)
    # df_data = df_data.sort_index()

    return df_data

In [94]:
# Operation between two dataframes with formats hours

COL_UNIFIED = "Total"

def unfied_hours(df):
    if COL_UNIFIED in df.columns:
        return df
    df_aux = df[[COL_DATE]].copy()
    df_aux[COL_UNIFIED] = df[COL_HOURS].sum(axis=1)
    return df_aux

def operation_between_df(df1, df2, operation, absolute=False, unified=False):
    columns = COL_HOURS
    if unified:
        df1 = unfied_hours(df1)
        df2 = unfied_hours(df2)
        columns = [COL_UNIFIED]

    common_dates = pd.merge(df1, df2, on=COL_DATE, how="inner", suffixes=('_1', '_2'))
    common_dates.fillna(0, inplace=True)

    result = common_dates[[COL_DATE]].copy()

    for hour in columns:
        if operation == "sum":
            result[hour] = common_dates[f"{hour}_1"]+common_dates[f"{hour}_2"]
        elif operation == "subs":
                result[hour] = common_dates[f"{hour}_1"]-common_dates[f"{hour}_2"]
        elif operation == "mult":
                result[hour] = common_dates[f"{hour}_1"]*common_dates[f"{hour}_2"]
        elif operation == "div":
                result[hour] = 100*common_dates[f"{hour}_1"]/common_dates[f"{hour}_2"]
        else:
            raise ValueError("Operation not supported")

    if absolute:
        result[columns] = result[columns].abs()

    return result

def calculate_toleration(df, gap):
    df[[COL_UNIFIED]] = df[[COL_UNIFIED]].apply(lambda x: print(x.values))
    return df

In [95]:
# Plot graphics

def plot_results(df_daily, df_month, title, title_1, title_2):
    # Calcular promedio y desviación estándar para cada DataFrame
    mean_daily = df_daily[COL_UNIFIED].mean()
    std_daily = df_daily[COL_UNIFIED].std()
    std_daily_percent = (std_daily / mean_daily) * 100  # Desviación estándar como porcentaje del promedio
    
    mean_month = df_month[COL_UNIFIED].mean()
    std_month = df_month[COL_UNIFIED].std()
    std_month_percent = (std_month / mean_month) * 100  # Desviación estándar como porcentaje del promedio
    
    # Crear la figura con subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=(title_1, title_2))

    # Gráfico de datos horarios
    fig.add_trace(
        go.Scatter(x=df_daily[COL_DATE], y=df_daily[COL_UNIFIED], mode='lines', name='Horario'),
        row=1, col=1
    )

    # Añadir anotación para promedio y desviación estándar en la esquina superior derecha del primer subplot
    fig.add_annotation(
        xref="x domain", yref="y domain", x=0.95, y=0.95, showarrow=False,
        text=f"X̅: {mean_daily:.2f}<br>σ: {std_daily_percent:.2f}%",
        row=1, col=1
    )

    # Gráfico de datos mensuales
    fig.add_trace(
        go.Scatter(x=df_month[COL_DATE], y=df_month[COL_UNIFIED], mode='lines', name='Mensual'),
        row=1, col=2
    )

    # Añadir anotación para promedio y desviación estándar en la esquina superior derecha del segundo subplot
    fig.add_annotation(
        xref="x domain", yref="y domain", x=0.95, y=0.95, showarrow=False,
        text=f"X̅: {mean_month:.2f}<br>σ: {std_month_percent:.2f}%",
        row=1, col=2
    )

    # Setear dimensiones y título de la figura
    fig.update_layout(height=400, width=1000, title_text=title)
    # Setear título del eje y en el primer subplot
    fig.update_yaxes(title_text="Desviación [%]", row=1, col=1)

    return fig

In [96]:
# read excel file
path = "./data/data_by_concept.xlsx"
# read with a context manager
excel_data = pd.ExcelFile(path)

In [97]:
gene_sheet_name = "Gene"
gen_prog_sheet_name = "GeneProgDesp"
gen_reprog_sheet_name = "GeneProgRedesp"

monthly_spot_price_name = "ProyeccionPreciosMensual"
hour_spot_price_name = "ProyeccionPreciosHorario"
monthly_gene_projection_name = "ProyeccionEnergia"

In [98]:
# open the sheet of generation in a dataframe
gen_real = read_each_sheet(gene_sheet_name, excel_data, "3IRX")
gen_desp = read_each_sheet(gen_prog_sheet_name, excel_data, "3IRX")
gen_redesp = read_each_sheet(gen_reprog_sheet_name, excel_data, "3IRX")

monthly_spot_price = read_each_sheet(monthly_spot_price_name, excel_data, index_col=None)
hour_spot_price = read_each_sheet(hour_spot_price_name, excel_data, index_col=None)
monthly_gene_projection = read_each_sheet(monthly_gene_projection_name, excel_data, index_col=None)

In [99]:
# Para el despacho

desv_desp = operation_between_df(gen_real, gen_desp, "subs", absolute=True , unified=True)
percentage_desv_desp = operation_between_df(desv_desp, gen_desp, "div", unified=True)

percentage_desv_desp_odds = percentage_desv_desp.copy()
percentage_desv_desp_odds[COL_UNIFIED] = percentage_desv_desp_odds[COL_UNIFIED].apply(
    lambda x: 1 if x > 15 else 0
)

percentage_desv_desp_penalty = percentage_desv_desp.copy()
percentage_desv_desp_penalty[COL_UNIFIED] = percentage_desv_desp_penalty[COL_UNIFIED].apply(
    lambda x: x if x > 15 else 0
)
percentage_desv_desp_penalty_rem0 = percentage_desv_desp_penalty[
    percentage_desv_desp_penalty[COL_UNIFIED] > 0
]

# Para el redespacho
df_desv_reprog = operation_between_df(gen_real, gen_redesp, "subs", absolute=True , unified=True)
percentage_desv_redesp = operation_between_df(df_desv_reprog, gen_redesp, "div", unified=True)

percentage_desv_redesp_odds = percentage_desv_redesp.copy()
percentage_desv_redesp_odds[COL_UNIFIED] = percentage_desv_redesp_odds[COL_UNIFIED].apply(
    lambda x: 1 if x > 8 else 0
)

percentage_desv_redesp_penalty = percentage_desv_redesp.copy()
percentage_desv_redesp_penalty[COL_UNIFIED] = percentage_desv_redesp_penalty[COL_UNIFIED].apply(
    lambda x: x if x > 8 else 0
)
percentage_desv_redesp_penalty_rem0 = percentage_desv_redesp_penalty[
    percentage_desv_redesp_penalty[COL_UNIFIED] > 0
]


In [100]:
monthly_avg_dev_desp_penalty = percentage_desv_desp_penalty_rem0.resample(
    'ME', on=COL_DATE)[COL_UNIFIED].mean().reset_index()
monthly_avg_dev_desp_odds = percentage_desv_desp_odds.resample(
    'ME', on=COL_DATE)[COL_UNIFIED].mean().reset_index()

monthly_avg_dev_desp = operation_between_df(
    monthly_avg_dev_desp_penalty,
    monthly_avg_dev_desp_odds,
    "mult",
    unified=True
)

monthly_avg_redesp_penalty = percentage_desv_redesp_penalty_rem0.resample(
    'ME', on=COL_DATE)[COL_UNIFIED].mean().reset_index()
monthly_avg_redesp_odds = percentage_desv_redesp_odds.resample(
    'ME', on=COL_DATE)[COL_UNIFIED].mean().reset_index()

monthly_avg_redesp = operation_between_df(
    monthly_avg_redesp_penalty,
    monthly_avg_redesp_odds,
    "mult",
    unified=True
)

mean_dev_desp_value = float(percentage_desv_desp_penalty[COL_UNIFIED].mean())
mean_dev_redesp_value = float(percentage_desv_redesp_penalty[COL_UNIFIED].mean())

mean_dev_desp_rem0_value = float(percentage_desv_desp_penalty_rem0[COL_UNIFIED].mean())
mean_dev_redesp_rem0_value = float(percentage_desv_redesp_penalty_rem0[COL_UNIFIED].mean())
mean_dev_desp_rem0_value
mean_dev_redesp_rem0_value

23.3033640506289

In [101]:

# Crear la figura con subplots
fig = plot_results(
    percentage_desv_desp_penalty, 
    monthly_avg_dev_desp, 
    "Desviación del Despacho", 
    "Desviación diaria de energía",
    "Desviación promedio mensual de energía", 
)
fig.show()

In [102]:
# fig_avg_reprog = px.line(monthly_avg_reprog, x=COL_DATE, y=COL_UNIFIED, title='Total mensual')
# fig_avg_reprog.update_layout(title='Desviación reprogramada', xaxis_title='Fecha', yaxis_title='Desviación (%)')

# Crear la figura con subplots
fig = plot_results(
    percentage_desv_redesp_penalty, 
    monthly_avg_redesp, 
    "Desviación Redespacho", 
    "Desviación diaria de energía",
    "Desviación promedio mensual de energía", 
)
fig.show()


In [103]:
# Crear la figura con subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Desviación despacho", "Desviación del redespacho"))

# Agregar el diagrama de bigotes para el primer DataFrame
fig.add_trace(
    go.Box(y=percentage_desv_desp_penalty[COL_UNIFIED], name="Despacho", boxpoints='all', jitter=0.3, pointpos=-1.8),
    row=1, col=1
)

# Agregar el diagrama de bigotes para el segundo DataFrame
fig.add_trace(
    go.Box(y=percentage_desv_redesp_penalty[COL_UNIFIED], name="Redespacho", boxpoints='all', jitter=0.3, pointpos=-1.8),
    row=1, col=2
)


In [152]:
def remove_outliers(df):
    q1 = df[COL_UNIFIED].quantile(0.25)
    q3 = df[COL_UNIFIED].quantile(0.75)
    iqr = q3 - q1
    return df[(df[COL_UNIFIED] > q1 - 1.5*iqr) & (df[COL_UNIFIED] < q3 + 1.5*iqr)]

def get_outliers(df):
    q1 = df[COL_UNIFIED].quantile(0.25)
    q3 = df[COL_UNIFIED].quantile(0.75)
    iqr = q3 - q1
    return df[(df[COL_UNIFIED] <= q1 - 1.5*iqr) | (df[COL_UNIFIED] >= q3 + 1.5*iqr)]

desv_desp_percentage_without_outliers = remove_outliers(percentage_desv_desp_penalty)
desv_redesp_percentage_without_outliers = remove_outliers(percentage_desv_redesp_penalty)

desv_desp_percentage_outliers = get_outliers(percentage_desv_desp_penalty)

monthly_avg_desp_without_outliers = desv_desp_percentage_without_outliers.resample(
    'ME', on=COL_DATE)[COL_UNIFIED].mean().reset_index()
monthly_avg_redesp_without_outliers = desv_redesp_percentage_without_outliers.resample(
    'ME', on=COL_DATE)[COL_UNIFIED].mean().reset_index()

mean_dev_desp_without_outliers = float(desv_desp_percentage_without_outliers[COL_UNIFIED].mean())
mean_dev_redesp_without_outliers = float(desv_redesp_percentage_without_outliers[COL_UNIFIED].mean())

mean_dev_desp_rem0_without_outliers = float(
    desv_desp_percentage_without_outliers[
        desv_desp_percentage_without_outliers[COL_UNIFIED] > 0
    ][COL_UNIFIED].mean()
)
prob_dev_desp_rem0_without_outliers = desv_desp_percentage_without_outliers[
    desv_desp_percentage_without_outliers[COL_UNIFIED] > 0
].shape[0] / percentage_desv_desp_penalty[
    percentage_desv_desp_penalty[COL_UNIFIED] > 0
].shape[0]
total_mean_dev_desp_rem0_without_outliers = mean_dev_desp_rem0_without_outliers*prob_dev_desp_rem0_without_outliers

mean_dev_desp_with_outliers = float(desv_desp_percentage_outliers[COL_UNIFIED].mean())
prob_dev_desp_with_outliers = 1 - prob_dev_desp_rem0_without_outliers
total_mean_dev_desp_with_outliers = mean_dev_desp_with_outliers*prob_dev_desp_with_outliers

print("Media de la desviación del despacho sin extremos: ", mean_dev_desp_rem0_without_outliers)
print("Probabilidad de valores medios sin extremos: ", prob_dev_desp_rem0_without_outliers)
print("Media total de la desviación del despacho sin extremos: ", total_mean_dev_desp_rem0_without_outliers)
print()
print("Media de la desviación del despacho con extremos: ", mean_dev_desp_with_outliers)
print("Probabilidad de desviación del despacho de los extremos: ", prob_dev_desp_with_outliers)
print("Media total de la desviación del despacho con extremos: ", total_mean_dev_desp_with_outliers)

total_prob = percentage_desv_desp_penalty[percentage_desv_desp_penalty[COL_UNIFIED] > 0].shape[0] / percentage_desv_desp_penalty.shape[0]


total_mean = (total_mean_dev_desp_rem0_without_outliers + total_mean_dev_desp_with_outliers) * total_prob
total_mean


Media de la desviación del despacho sin extremos:  27.90404297658373
Probabilidad de valores medios sin extremos:  0.8446601941747572
Media total de la desviación del despacho sin extremos:  23.569434358861987

Media de la desviación del despacho con extremos:  316.7547464157196
Probabilidad de desviación del despacho de los extremos:  0.15533980582524276
Media total de la desviación del despacho con extremos:  49.20462080244189


31.761557972941947

In [156]:
fig = plot_results(
    desv_desp_percentage_without_outliers, 
    monthly_avg_desp_without_outliers, 
    "Desviación Despacho sin extremos", 
    "Desviación diaria de energía",
    "Desviación promedio mensual de energía", 
)
# fig.show()
desv_desp_percentage_without_outliers.to_excel("desv_desp_percentage_without_outliers.xlsx")

In [106]:
fig = plot_results(
    desv_redesp_percentage_without_outliers, 
    monthly_avg_redesp_without_outliers, 
    "Desviación Redespacho sin outliers", 
    "Desviación diaria de energía",
    "Desviación promedio mensual de energía", 
)
fig.show()

In [107]:
gen_real_by_day = unfied_hours(gen_real)
# plot a box plot of the data gen_real_by_day
fig = px.box(gen_real_by_day, y=COL_UNIFIED, title='Despacho')
fig.update_traces(jitter=0.3, pointpos=-1.8, boxpoints='all')
fig.update_layout(title='Distribución Generación Real diaria', yaxis_title='Energía [kWh]')

In [115]:
gen_real_and_deviation_by_day = pd.merge(
    gen_real_by_day, 
    percentage_desv_desp_penalty, 
    on=COL_DATE, 
    how='inner'
)
gen_real_and_deviation_by_day["Total_x"] = gen_real_and_deviation_by_day["Total_x"]/1000

fig = px.scatter(
    gen_real_and_deviation_by_day, 
    x='Total_x', 
    y='Total_y', 
    hover_data=[COL_DATE]
)
fig.update_traces(
    marker=dict(size=10),
    selector=dict(mode='markers')
)

fig.update_layout(
    title="Energía MWh-dia vs Desviación",
    xaxis_title="Energía MWh-dia",
    yaxis_title="Desviación %",
    hovermode="closest"
)
# gen_real_and_deviation_by_day

In [187]:
monthly_gen_real_mean = gen_real.resample('ME', on=COL_DATE).mean().reset_index()

# Crear la figura
fig = go.Figure()
for _, row in monthly_gen_real_mean.iterrows():
    date = row[COL_DATE].strftime('%Y-%m-%d')
    fig.add_trace(
        go.Scatter(
            x=[f"{i:02}" for i in range(1,25)], 
            y=row[COL_HOURS], 
            mode='lines+markers', 
            name=date
        )
    )

# add trace with the mean of the all values of gen_real
fig.add_trace(
    go.Scatter(
        x=[f"{i:02}" for i in range(1,25)], 
        y=gen_real.mean()[COL_HOURS], 
        mode='lines+markers', 
        name='Promedio'
    )
)

fig.update_layout(
    title='Promedio mensual de la generación diaría', 
    xaxis_title='Hora', 
    yaxis_title='Energía [kWh]'
)

In [48]:
gen_real_mean = gen_real.fillna(0).mean()
gen_real_mean_dict = gen_real_mean.to_dict()
gen_real_mean_dict.pop(COL_DATE)

gen_real_mean_dict_pu = {k: v/sum(gen_real_mean_dict.values()) for k, v in gen_real_mean_dict.items()}
gen_real_mean_dict_pu

{'Values_Hour01': 0.0,
 'Values_Hour02': 0.0,
 'Values_Hour03': 0.0,
 'Values_Hour04': 0.0,
 'Values_Hour05': 0.0,
 'Values_Hour06': 1.5013577576737743e-05,
 'Values_Hour07': 0.010196521832251148,
 'Values_Hour08': 0.042916077090148406,
 'Values_Hour09': 0.07482048202801195,
 'Values_Hour10': 0.10221924155041047,
 'Values_Hour11': 0.1213514132997428,
 'Values_Hour12': 0.12883756653866954,
 'Values_Hour13': 0.13014773853093248,
 'Values_Hour14': 0.12727467582586693,
 'Values_Hour15': 0.1135857471366739,
 'Values_Hour16': 0.08735591851668942,
 'Values_Hour17': 0.05029774398677142,
 'Values_Hour18': 0.010975683892000255,
 'Values_Hour19': 6.1761942547592955e-06,
 'Values_Hour20': 0.0,
 'Values_Hour21': 0.0,
 'Values_Hour22': 0.0,
 'Values_Hour23': 0.0,
 'Values_Hour24': 0.0}

In [67]:
# ignore SettingWithCopyWarning
pd.options.mode.chained_assignment = None

monthly_spot_price 
# monthly_gene_projection 
COL_SPOT_PRICE = ["Promedio", "Alta", "Baja"]

hour_spot_price_normalized = hour_spot_price.copy()
# merge with monthly_spot_price
hour_spot_price_normalized = pd.merge(
    hour_spot_price_normalized, monthly_spot_price, on=COL_DATE, how="inner"
)
for col in COL_HOURS:
    hour_spot_price_normalized[col] = hour_spot_price_normalized[col] / hour_spot_price_normalized[COL_SPOT_PRICE[0]]

hour_spot_price_promedio = hour_spot_price_normalized[[COL_DATE]+COL_HOURS]
for col in COL_HOURS:
    hour_spot_price_promedio[col] = hour_spot_price_promedio[col] * hour_spot_price_normalized[COL_SPOT_PRICE[0]]

hour_spot_price_alta = hour_spot_price_normalized[[COL_DATE]+COL_HOURS]
for col in COL_HOURS:
    hour_spot_price_alta[col] = hour_spot_price_alta[col] * hour_spot_price_normalized[COL_SPOT_PRICE[1]]

hour_spot_price_baja = hour_spot_price_normalized[[COL_DATE]+COL_HOURS]
for col in COL_HOURS:
    hour_spot_price_baja[col] = hour_spot_price_baja[col] * hour_spot_price_normalized[COL_SPOT_PRICE[2]]
